# basic cnn from scratch using pytorch lightning


In [ ]:
!pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms,datasets
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np
import pandas as pd
import cv2
import zipfile
reprocess_data = True
from PIL import Image
from torchsummary import summary
import time
from torch.utils.tensorboard import SummaryWriter
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")
import seaborn as sns
import pytorch_lightning as pl
from sklearn.preprocessing import LabelEncoder
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

In [ ]:
wandb.login(key=secret_value_0)

In [ ]:
print("gpu available: ",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ",device)

In [ ]:
label_df  =pd.read_csv("../input/dog-breed-identification/labels.csv")
label_df.head()

In [ ]:
label_df.shape

In [ ]:
num_labels = len(list(label_df["breed"].value_counts()))
print("num of labels is : ",num_labels)

In [ ]:
train_filelist = label_df["id"]+".jpg"
train_filepath  = "../input/dog-breed-identification/train"
test_filepath = "../input/dog-breed-identification/test"
test_filelist = os.listdir(test_filepath)
le = LabelEncoder()
train_lables = le.fit_transform(label_df["breed"])

In [ ]:
class Dataset():
    def __init__(self,dataset_type,filelist,filepath,labels = None,transform = None):
        self.dataset_type = dataset_type
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return int(len(self.filelist))
    def __getitem__(self,index):
        imgpath = os.path.join(self.filepath,self.filelist[index])
        img = Image.open(imgpath)
        label = self.labels[index]
        if self.transform is not None:
            img = self.transform(img)
        if self.dataset_type == "train":    
            return img,label
        else:
            return img 

In [ ]:
transformations = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])

In [ ]:
train = Dataset("train",train_filelist,train_filepath,train_lables,transformations)
test =  Dataset("test",test_filelist,test_filepath,transformations)

In [ ]:
train_set,val_set = torch.utils.data.random_split(train,[round(len(train)*0.8),round(len(train)*0.2)]) 

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_set,batch_size = 64,num_workers = 2)
val_loader = torch.utils.data.DataLoader(dataset = val_set,batch_size = 64,num_workers = 2)

In [ ]:
class doggy_vgg_16(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.train_accuracy = pl.metrics.Accuracy()
        self.val_accuracy = pl.metrics.Accuracy()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3,64,3,padding= 1),
            nn.ReLU(),
#             nn.MaxPool2d(2,2),
#             nn.Dropout(0.1),
            nn.Conv2d(64,64,3,padding= 1),
            nn.ReLU(),
            nn.MaxPool2d((2,2))            
            ) 
            
        self.conv2 =   nn.Sequential(
            nn.Conv2d(64,128,3,padding= 1),
            nn.ReLU(),
#             nn.MaxPool2d(2,2),
#             nn.Dropout(0.1),
            nn.Conv2d(128,128,3,padding= 1),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
        ) 
        self.conv3 =   nn.Sequential(
            nn.Conv2d(128,256,3,padding= 1),
            nn.ReLU(),
            nn.Conv2d(256,256,3,padding= 1),
            nn.ReLU(),
            nn.Conv2d(256,256,3,padding= 1),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
            )
        self.conv4 =   nn.Sequential(
            nn.Conv2d(256,512,3,padding= 1),
            nn.ReLU(),
            nn.Conv2d(512,512,3,padding= 1),
            nn.ReLU(),
            nn.Conv2d(512,512,3,padding= 1),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
            )
        self.conv5 =   nn.Sequential(
            nn.Conv2d(512,512,3,padding= 1),
            nn.ReLU(),
            nn.Conv2d(512,512,3,padding= 1),
            nn.ReLU(),
            nn.Conv2d(512,512,3,padding= 1),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
            )
        self.fc1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512*7*7,4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096,1000)
        )
            
        self.fc2 = nn.Sequential(
        nn.Linear(1000,120),
        )

    def forward(self,x):
        x= self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.softmax(x,dim = 1) 
    
    #pytorch lighning functions
    def configure_optimizers(self):
        optimiser = torch.optim.Adam(self.parameters(),lr=0.00001)
        return optimiser
    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        loss = F.cross_entropy(pred, y)
        self.train_accuracy(pred,y)
        self.log('train_acc_epoch', self.train_accuracy,on_step=False, on_epoch=True,prog_bar=True)
        return loss 
#     def training_epoch_end(self,output):
#         self.log('train_acc_epoch', self.train_accuracy(),on_step=True, on_epoch=True)
    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        val_loss = F.cross_entropy(pred, y)
        self.val_accuracy(pred,y)
        self.log('val_acc_epoch', self.val_accuracy,on_step=False, on_epoch=True,prog_bar=True)
        return val_loss
#     def validation_epoch_end(self,outputs):
#         self.log('train_acc_epoch', self.train_accuracy(),on_step=True, on_epoch=True)
        
    
    

In [ ]:
wandb.init(project="cnn_pytorch_lightning_dog_breeds",reinit=True)

In [ ]:
model = doggy_vgg_16()
model.to(device)
summary(model,(3,224,224))

In [ ]:
wandb_logger = WandbLogger()
wandb_logger.watch(model, log='gradients', log_freq=100)

In [ ]:
trainer = Trainer(gpus=1,max_epochs = 20,logger=wandb_logger)
trainer.fit(model,train_loader,val_loader)

In [ ]:
wandb.finish()